# CNN（畳み込みニューラルネットワーク）の基礎

このノートブックでは、画像分類の数理的基礎を学びます。

## 目次
1. 画像データの構造
2. なぜCNNが必要か？（特徴量の保持）
3. CNNの全体像
4. 各層の役割
5. 手書き数字分類（MNIST）

## 1. 画像データの構造

画像データは**ピクセル**（画素）と呼ばれる微小な四角形の集合体です。

| 画像タイプ | データ構造 | ピクセル値 |
|-----------|-----------|----------|
| グレースケール | 2次元 (H, W) | 0（黒）〜 255（白）|
| カラー（RGB） | 3次元 (H, W, 3) | 各チャンネル 0〜255 |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 図3.2: グレースケール画像のピクセル値の例（教科書より）
# 0〜255の値で濃淡を表現
pixel_example = np.array([
    [0,   85,  170],
    [255, 50,  100],
    [150, 200, 225]
], dtype=np.uint8)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 左: ピクセル値を数値で表示
ax1 = axes[0]
ax1.imshow(pixel_example, cmap='gray', vmin=0, vmax=255)
for i in range(3):
    for j in range(3):
        ax1.text(j, i, str(pixel_example[i, j]), 
                ha='center', va='center', fontsize=16,
                color='white' if pixel_example[i, j] < 128 else 'black')
ax1.set_title('グレースケール画像のピクセル値\n(0=黒, 255=白)', fontsize=12)
ax1.axis('off')

# 右: グラデーションバー
ax2 = axes[1]
gradient = np.linspace(0, 255, 256).reshape(1, -1).astype(np.uint8)
ax2.imshow(np.repeat(gradient, 50, axis=0), cmap='gray', vmin=0, vmax=255)
ax2.set_title('ピクセル値と濃淡の対応', fontsize=12)
ax2.set_xlabel('0 (黒) ←――――――→ 255 (白)')
ax2.set_yticks([])

plt.tight_layout()
plt.show()

## 2. なぜCNNが必要か？（特徴量の保持）

### 問題：画像データを直接1次元に変換すると情報が失われる

画像分類の最終段階では**ソフトマックス関数**を使いますが、この関数は**1次元データ**を必要とします。

しかし、画像データは2次元（または3次元）構造を持っており、単純に1次元に変換すると**重要な特徴量**を失う可能性があります。

### 特徴量とは？

**特徴量**とは、画像が持つ意味のある情報です：
- ピクセルの数値そのもの
- ピクセル集合同士の**近さ/遠さ**（空間的関係）

**例：顔認識**
- 人の顔は「目、鼻、口」が特徴
- これらは2次元平面で**一定の距離内に集まっている**
- 目、鼻、口は**上から下に並んでいる**

→ この空間的な位置関係こそが重要な特徴量！

In [ ]:
# 特徴量の重要性を示す例
# 同じピクセル値でも配置が違えば全く別の画像になる

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# 元の画像（8の形）
original = np.array([
    [0, 1, 1, 1, 0],
    [1, 0, 0, 0, 1],
    [0, 1, 1, 1, 0],
    [1, 0, 0, 0, 1],
    [0, 1, 1, 1, 0]
]) * 255

# 1次元に変換して再配置（ランダム）
flat = original.flatten()
np.random.seed(42)
shuffled = flat.copy()
np.random.shuffle(shuffled)
randomized = shuffled.reshape(5, 5)

axes[0].imshow(original, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('元の画像「8」\n（空間構造あり）', fontsize=11)
axes[0].axis('off')

axes[1].bar(range(25), flat, color='steelblue')
axes[1].set_title('1次元に変換\n（空間情報が失われる）', fontsize=11)
axes[1].set_xlabel('インデックス')
axes[1].set_ylabel('ピクセル値')

axes[2].imshow(randomized, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('シャッフル後\n（同じピクセル値、違う意味）', fontsize=11)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("→ 単純に1次元化すると、形状（空間的特徴）の情報が失われる！")

## 3. CNNの全体像

CNNは**形状を保持しながら**特徴を抽出し、最終的に1次元に変換します。

```
入力層 → [畳み込み層 → プーリング層] × N → 全結合層 → 出力層
  ↓           ↓              ↓              ↓          ↓
 画像      特徴抽出       サイズ圧縮      1次元化    確率出力
(2D/3D)    (2D/3D)        (2D/3D)        (1D)       (1D)
```

**ポイント**: 畳み込みとプーリングを繰り返すことで、空間構造を保ちながら徐々に特徴を圧縮していく

### 具体的なCNN構成例（図3.4）

手書き数字（0〜9）を分類するCNNの例：

| 層 | フィルタ/サイズ | ストライド | 出力サイズ | 説明 |
|---|--------------|---------|---------|------|
| **入力層** | - | - | 24×24 | グレースケール画像 |
| **畳み込み層①** | 5×5 × 2枚 | 1 | 20×20×2 | 2種類の特徴マップ生成 |
| **プーリング層①** | 2×2 | 2 | 10×10×2 | サイズを半分に圧縮 |
| **畳み込み層②** | 3×3 × 8枚 | 1 | 8×8×4 | より複雑な特徴を抽出 |
| **プーリング層②** | 2×2 | 2 | 4×4×4 | さらに圧縮 |
| **全結合層** | - | - | 64×1 | 4×4×4=64を1次元化 |
| **出力層** | ソフトマックス | - | 10 | 0〜9の確率 |

### 用語解説
- **フィルタ**: 特徴を抽出するための重み行列（カーネルとも呼ぶ）
- **ストライド**: フィルタを動かす幅
  - stride=1 → 1ピクセルずつ移動
  - stride=2 → 2ピクセルずつ移動（出力サイズが半分に）

In [ ]:
# 図3.4: CNNの具体的な構成例を可視化
# 各層の出力サイズの変化を追跡

def calculate_conv_output(input_size, filter_size, stride=1, padding=0):
    """畳み込み層の出力サイズを計算"""
    return (input_size - filter_size + 2 * padding) // stride + 1

def calculate_pool_output(input_size, pool_size, stride):
    """プーリング層の出力サイズを計算"""
    return (input_size - pool_size) // stride + 1

# CNNの各層の構成
layers = [
    {"name": "入力層", "size": (24, 24, 1), "color": "lightgray"},
    {"name": "畳み込み層①\n5×5, stride=1", "size": (20, 20, 2), "color": "steelblue"},
    {"name": "プーリング層①\n2×2, stride=2", "size": (10, 10, 2), "color": "lightblue"},
    {"name": "畳み込み層②\n3×3, stride=1", "size": (8, 8, 4), "color": "steelblue"},
    {"name": "プーリング層②\n2×2, stride=2", "size": (4, 4, 4), "color": "lightblue"},
    {"name": "全結合層", "size": (64, 1, 1), "color": "orange"},
    {"name": "出力層\nソフトマックス", "size": (10, 1, 1), "color": "green"},
]

fig, ax = plt.subplots(figsize=(14, 6))

x_pos = 0
for i, layer in enumerate(layers):
    h, w, c = layer["size"]
    # 表示用のスケーリング
    display_h = max(h / 3, 0.8)
    display_w = max(w / 3, 0.8)
    
    rect = plt.Rectangle((x_pos, 3 - display_h/2), display_w, display_h, 
                          facecolor=layer["color"], edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    
    # サイズラベル
    if c > 1:
        size_text = f"{h}×{w}×{c}"
    else:
        size_text = f"{h}×{w}" if w > 1 else f"{h}"
    ax.text(x_pos + display_w/2, 3 - display_h/2 - 0.3, size_text, 
            ha='center', va='top', fontsize=9)
    
    # 層の名前
    ax.text(x_pos + display_w/2, 3 + display_h/2 + 0.2, layer["name"], 
            ha='center', va='bottom', fontsize=8)
    
    # 矢印
    if i < len(layers) - 1:
        ax.annotate('', xy=(x_pos + display_w + 0.3, 3), 
                   xytext=(x_pos + display_w + 0.1, 3),
                   arrowprops=dict(arrowstyle='->', color='black'))
    
    x_pos += display_w + 0.5

ax.set_xlim(-0.5, x_pos)
ax.set_ylim(0, 6)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('図3.4: CNNの構成例（手書き数字分類）', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# 出力サイズの計算を確認
print("出力サイズの計算:")
print(f"  入力: 24×24")
print(f"  畳み込み①: (24-5)/1+1 = 20 → 20×20×2")
print(f"  プーリング①: (20-2)/2+1 = 10 → 10×10×2")
print(f"  畳み込み②: (10-3)/1+1 = 8 → 8×8×4")
print(f"  プーリング②: (8-2)/2+1 = 4 → 4×4×4")
print(f"  全結合: 4×4×4 = 64 → 64×1")
print(f"  出力: 10クラス（0〜9）")

## 4. 各層の役割

| 層 | 処理内容 | 入出力 |
|---|---------|-------|
| **入力層** | 画像データを受け取る | → 2D/3D |
| **畳み込み層** | フィルタで積和演算、特徴を抽出 | 2D/3D → 特徴マップ |
| **プーリング層** | 特徴マップのサイズを縮小（データ量削減） | 特徴マップ → 小さい特徴マップ |
| **全結合層** | すべての特徴量を1次元に配列 | 2D/3D → 1D |
| **出力層** | ソフトマックス関数で確率を出力 | 1D → 分類結果 |

### CNNの名前の由来
- **C**NN = **C**onvolutional Neural Network
- **Convolutional** = 畳み込み

In [ ]:
# CNNの処理フローを視覚化
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# 1. 入力画像
input_img = np.random.randint(0, 255, (28, 28))
axes[0].imshow(input_img, cmap='gray')
axes[0].set_title('入力層\n(28×28)', fontsize=10)
axes[0].axis('off')

# 2. 畳み込み後
conv1 = np.random.randint(0, 255, (24, 24))
axes[1].imshow(conv1, cmap='Blues')
axes[1].set_title('畳み込み層\n(24×24)\n特徴抽出', fontsize=10)
axes[1].axis('off')

# 3. プーリング後
pool1 = np.random.randint(0, 255, (12, 12))
axes[2].imshow(pool1, cmap='Blues')
axes[2].set_title('プーリング層\n(12×12)\nサイズ圧縮', fontsize=10)
axes[2].axis('off')

# 4. 全結合層
fc = np.random.rand(1, 50)
axes[3].imshow(fc, cmap='Oranges', aspect='auto')
axes[3].set_title('全結合層\n(1×N)\n1次元化', fontsize=10)
axes[3].axis('off')

# 5. 出力層
output = np.zeros(10)
output[8] = 0.95  # "8"の確率が高い
axes[4].bar(range(10), output, color='green')
axes[4].set_title('出力層\n(確率分布)\nソフトマックス', fontsize=10)
axes[4].set_xticks(range(10))
axes[4].set_xlabel('クラス (0-9)')

plt.tight_layout()
plt.show()

## 5. 手書き数字分類（MNIST）

MNISTデータセット：28×28ピクセルのグレースケール手書き数字（0〜9）

In [ ]:
# MNISTデータセットの読み込み
import torch
from torchvision import datasets, transforms

# データの前処理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST用の正規化
])

# データセットのダウンロード
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

print(f"訓練データ: {len(train_dataset)} 枚")
print(f"テストデータ: {len(test_dataset)} 枚")
print(f"画像サイズ: {train_dataset[0][0].shape}")

In [ ]:
# サンプル画像の表示
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    image, label = train_dataset[i]
    ax.imshow(image.squeeze(), cmap='gray')
    ax.set_title(f'ラベル: {label}')
    ax.axis('off')
plt.suptitle('MNISTデータセットのサンプル', fontsize=14)
plt.tight_layout()
plt.show()

## まとめ

1. **画像データ**はピクセルの2D/3D配列で、0〜255の値で濃淡/色を表現
2. **特徴量**（空間的な位置関係）を保持することが画像認識で重要
3. **CNN**は畳み込み→プーリングを繰り返し、形状を保ちながら特徴を抽出
4. 最終的に**全結合層**で1次元化し、**ソフトマックス**で確率を出力

## 次のステップ

次のノートブックでは、**畳み込み演算**の数理について詳しく学びます。